In [1]:
import numpy as np
import h5py
import glob
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import logging


import sys
import os
sys.path.append(os.getcwd() + '/..')
import sib_ldsc_z as ld

In [69]:
traitnos = glob.glob("/disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/*/")
traitnos = [int(i[-3:-1].strip('/')) for i in traitnos]

traitcodes = pd.read_csv("/disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/traits.txt", 
            names = ['traitcode', 'trait'], 
            sep = ' ')
            
traitcode = 7
traitname = traitcodes.loc[traitcodes['traitcode'] == traitcode, 'trait'].values[0]
print(traitname)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [70]:
startTime = datetime.datetime.now()  
logging.info(f"Start time:  {startTime}")

# == Reading in Data == #
print("=====================================")
print("Reading in Data")
print("=====================================")
# reading in  data
files = glob.glob(f"/disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/{traitcode}/chr_*.hdf5")

file = files[0]
print("Reading in file: ", file)
hf = h5py.File(file, 'r')
metadata = hf.get("bim")[()]
chromosome = metadata[:, 0]
snp = metadata[:, 3]
theta  = hf.get('estimate')[()]
se  = hf.get('estimate_ses')[()]
N = hf.get('N_L')[()]
S = hf.get('estimate_covariance')[()]
f = hf.get('freqs')[()]

# normalizing S
sigma2 = hf.get('sigma2')[()]
tau = hf.get('tau')[()]
phvar = sigma2+sigma2/tau


for file in files[1:]:
    print("Reading in file: ", file)
    hf = h5py.File(file, 'r')
    metadata = hf.get("bim")[()]
    chromosome_file = metadata[:, 0]  
    snp_file = metadata[:, 3]
    theta_file  = hf.get('estimate')[()]
    se_file  = hf.get('estimate_ses')[()]
    S_file = hf.get('estimate_covariance')[()]
    f_file = hf.get('freqs')[()]
    N_file = hf.get('N_L')[()]

    # normalizing S
    sigma2 = hf.get('sigma2')[()]
    tau = hf.get('tau')[()]

    chromosome = np.append(chromosome, chromosome_file, axis = 0)
    snp = np.append(snp, snp_file, axis = 0)
    theta = np.append(theta, theta_file, axis = 0)
    se = np.append(se, se_file, axis = 0)
    S = np.append(S, S_file, axis = 0)
    f = np.append(f, f_file, axis = 0)
    N = np.append(N, N_file, axis = 0)

# Constructing dataframe of data
zdata = pd.DataFrame({'CHR' : chromosome,
                    'BP' : snp,
                    'N' : N,
                    "f" : f,
                    'theta' : theta.tolist(),
                    'se' : se.tolist(),
                    "S" : S.tolist()})


zdata['CHR'] = zdata['CHR'].astype(int)
zdata['BP'] = zdata['BP'].astype(str).str.replace("b'", "").str[:-1]

Reading in Data
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_2.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_6.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_19.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_15.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_20.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_11.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_14.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_21.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_10.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_3.hdf5
Reading in file:  /disk/genetics/ukb/alextisyoung/GS20k_sumstats/traits/7/chr_7.hdf5
Reading in file:  /disk/genetics/ukb/alext

In [71]:
# == Reading in LD Scores == #

ldscore_path = "/disk/genetics/ukb/alextisyoung/GS20k_sumstats/ldscores/"
ldfiles = "*[0-9].l2.ldscore.gz"
ldcolnames = ["CHR", "SNP", "BP", "L2"]
Mfiles = "*[0-9].l2.M_5_50"
Mcolnames = ["M", "CHR"]

ldscores, mfile = ld.read_ldscores(ldscore_path, ldfiles, ldcolnames, Mfiles, Mcolnames)

In [72]:
# Merging LD scores with main Data Frame
ldscores['BP'] = ldscores['BP'].astype('int')
zdata['BP'] = zdata['BP'].astype('int')

main_df = zdata.merge(ldscores, how = "inner", on = ["CHR", "BP"])

# dropping NAs
main_df = main_df.dropna()

In [74]:
# transforming inputs

S = np.array(list(main_df.S)) 
theta = np.array(list(main_df.theta))
f = np.array(list(main_df["f"]))
l = np.array(list(main_df["L2"]))
u = np.array(list(main_df["L2"]))

# M = mfile['M'].sum()
M = len(S)

effect_estimated = "direct_plus_population"

S, theta = ld.transform_estimates(effect_estimated, S, theta)

In [78]:
# making z value
zval = ld.theta2z(theta, S, M = M)

# == Initializing model == #
model = ld.sibreg(S = S, 
                z = zval, 
                l = l,
                f = f,
                u = u,
                M = M) 

In [79]:
output_matrix, result = model.solve() 

No initial guess provided.
Making Method of Moments Guess
Initial estimate: [ 3.70066400e+01  5.01955328e+01 -9.91498853e-03]


ZeroDivisionError: division by zero

In [80]:
# manual optimization
S_hat = np.mean(S, axis = 0)
Dmat = ld.makeDmat(S_hat, M)
Snew = Dmat @ S_hat @ Dmat
z_var = np.cov(zval.T)
Vnew_init = z_var - Snew
V_init = np.linalg.inv(Dmat) @ Vnew_init @ np.linalg.inv(Dmat)
est_init = ld.Vmat2V(V_init, M)

In [81]:
est_init

array([ 3.70066400e+01,  5.01955328e+01, -9.91498853e-03])

In [83]:
from scipy.optimize import minimize
result = minimize(
    ld.neglike_wrapper, 
    est_init,
    jac = True,
    args = (zval, S, l, u, f, M),
    bounds = [(1e-6, None), (1e-6, None), (-1, 1)],
    method = 'L-BFGS-B'
)

In [86]:
result.x

array([1.00000000e-06, 4.49367194e-01, 9.99999999e-01])